## 1. 必要なライブラリのインポート

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## 2. データセットの準備 (ダミーデータ)

この例では、MovieLens-100K データセットに似たダミーデータを使用します。ユーザーのテキスト特徴は単純な文字列とします。

In [8]:
num_users = 100
num_items = 50


# ユーザーのテキスト特徴 (例: 趣味、自己紹介など)
user_texts = [f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)]

# ユーザーアイテムインタラクション (implicit feedback)
# ユーザーID, アイテムID, 評価 (1: インタラクションあり, 0: なし)
# 簡単のため、ランダムなインタラクションを生成
interactions = []
for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:  # 約30%の確率でインタラクションあり
            interactions.append([u_id, i_id, 1])
        else:
            interactions.append([u_id, i_id, 0])

interactions = torch.tensor(interactions, dtype=torch.float32)


# データローダーの作成
dataset = TensorDataset(interactions[:, 0].long(), interactions[:, 1].long(), interactions[:, 2])
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)


print("datasetの内容を確認")
for i in range(10):
    print(dataset[i])
print(f"{len(dataset)=}")

print("dataloaderの内容を確認")
for batch in dataloader:
    break  # 最初のバッチだけ確認
print(f"{len(dataloader)=}\n" ,f"{len(batch)=}\n", f"{batch=}")

datasetの内容を確認
(tensor(0), tensor(0), tensor(1.))
(tensor(0), tensor(1), tensor(0.))
(tensor(0), tensor(2), tensor(0.))
(tensor(0), tensor(3), tensor(0.))
(tensor(0), tensor(4), tensor(1.))
(tensor(0), tensor(5), tensor(0.))
(tensor(0), tensor(6), tensor(0.))
(tensor(0), tensor(7), tensor(0.))
(tensor(0), tensor(8), tensor(0.))
(tensor(0), tensor(9), tensor(0.))
len(dataset)=5000
dataloaderの内容を確認
len(dataloader)=100
 len(batch)=3
 batch=[tensor([17, 14,  0, 13, 85, 26, 59, 50, 38, 21, 19, 59, 17,  8, 76, 34, 88, 61,
        30, 54, 42,  2, 46, 95,  0,  4, 10, 90, 38, 49, 62, 57, 46, 48, 80, 28,
        68, 82, 95, 58, 12, 22, 84, 68,  9, 60, 36, 12, 93, 44]), tensor([ 0,  0, 19, 45, 38, 49,  6, 36, 24, 46, 33, 29, 13, 17, 39, 38,  6,  9,
        36, 36, 11, 37, 13, 45, 44, 30, 40, 13, 44, 21, 40, 46, 44, 22, 47, 33,
        21, 12, 30,  4, 28, 18, 21,  7, 23, 23, 31, 43, 20, 25]), tensor([0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 1.,

## 3. 軽量 LLM 埋め込みモデルのロード
Hugging Face の all-MiniLM-L6-v2 を使用します。これは軽量で、文の埋め込みに適しています。

In [9]:
# Hugging Face の軽量 LLM 埋め込みモデルのロード
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

# PLMの埋め込み次元を取得
plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")

PLM embedding dimension: 384


## 4. モデルの定義 (Transformer Block と PLM / PromptOutputLayer を線形層で代替)
ここでは、論文の図2 (Client Model) を参考に、以下の点を変更してモデルを定義します。

- **Joint Embedding Layer**: PLM と PromptOutputLayer の代わりに、直接 Hugging Face の LLM 埋め込みモデルを使用し、その出力に線形層を適用します。ユーザーのテキスト特徴から直接ユーザー埋め込みを生成します。

- **Transformer Block**: 単純な線形層に置き換えます。ユーザー埋め込みとアイテム埋め込みを結合し、予測を行います。

In [ ]:
class SimpleUFGraphFR(nn.Module):
    def __init__(
        self,
        num_users: int,
        num_items: int,
        item_embedding_dim: int,
        plm_model,
        plm_embedding_dim: int,
        joint_embedding_output_dim: int
    ):
        super(SimpleUFGraphFR, self).__init__()
        self.plm_model = plm_model
        self.item_embedding = nn.Embedding(num_items, item_embedding_dim)

        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)

        # ユーザ埋め込みとアイテム埋め込みを結合して予測する.
        self.prediction_layer = nn.Linear(joint_embedding_output_dim + item_embedding_dim, 1)

    def forward(self, user_ids, item_ids, user_texts_batch):
        # =========================
        # まずはユーザ埋め込みを作成する

        # NOTE バッチ毎にテキストを処理する
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')

        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :] # [CLS]トークンの埋め込みを使用

        # joint embedding layerの線形変換
        user_embedding = self.user_joint_embedding_linear(plm_output)  # (batch_size, joint_embedding_output_dim)
        # ユーザ埋め込み作成完了
        # =========================

        # =========================
        # アイテム埋め込みを取得
        item_embedding = self.item_embedding(item_ids)  # (batch_size, item_embedding_dim)
        # アイテム埋め込み完了
        # =========================


        # ユーザー埋め込みとアイテム埋め込みを結合
        combined_features = torch.cat((user_embedding, item_embedding), dim=1)
        # 予測
        prediction = torch.sigmoid(self.prediction_layer(combined_features))
        return prediction, self.user_joint_embedding_linear.weight # 線形層の重みを返す

## 5. モデルの初期化と学習ループ
このステップでは、グラフ構築やサーバー側での集約は行わず、各クライアント（この単純な実装では全体で1つのモデル）がローカルで学習する形式とします。

In [11]:
item_embedding_dim = 32
joint_embedding_output_dim = 100 # NOTE: 論文では、32だった.

# モデルのインスタンス化
model = SimpleUFGraphFR(num_users, num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim)

# オプティマイザと損失関数
optimizer = optim.Adam(model.parameters(), lr=0.001)
# NOTE:
# PyTorchで分類問題での損失関数は一般的には、nn.BCEWithLogitsLoss()を使うが、
# 今回は、予測層でsigmoidを使っているのでBCELossを利用する。
criterion = nn.BCELoss() 

# 学習ループ
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0.0
    for user_ids_batch, item_ids_batch, labels_batch in dataloader:
        # バッチ内のユーザIDに対応するテキスト特徴を取得
        current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]

        # 勾配初期化
        optimizer.zero_grad()
        predictions, _ = model(user_ids_batch, item_ids_batch, current_user_texts)

        # 損失計算
        loss = criterion(predictions.squeeze(), labels_batch)

        # 勾配計算とパラメータ更新
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")


Epoch 1/10, Loss: 0.6218
Epoch 2/10, Loss: 0.6172
Epoch 3/10, Loss: 0.6141
Epoch 4/10, Loss: 0.6121
Epoch 5/10, Loss: 0.6134
Epoch 6/10, Loss: 0.6108
Epoch 7/10, Loss: 0.6105
Epoch 8/10, Loss: 0.6104
Epoch 9/10, Loss: 0.6102
Epoch 10/10, Loss: 0.6113
